In [91]:
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from tqdm import tqdm

torch.manual_seed(42)

# Data preprocessing

In [2]:
# load data
df = pd.DataFrame()

for path in glob.glob("data/*.RData"):
    _df = pyreadr.read_r(path)
    k = list(_df.keys())[0]
    _df =  _df[k]
    df = pd.concat([df, _df])

df = df.reset_index()
df

,index,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0,0.0,1.0,1,0.25171,3672.4,4466.3,9.5122,27.057,42.473,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,1,0.0,1.0,2,0.25234,3642.2,4568.7,9.4145,26.999,42.586,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,2,0.0,1.0,3,0.24840,3643.1,4507.5,9.2901,26.927,42.278,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,3,0.0,1.0,4,0.25153,3628.3,4519.3,9.3347,26.999,42.330,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,4,0.0,1.0,5,0.21763,3655.8,4571.0,9.3087,26.901,42.402,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15329995,4999995,20.0,500.0,496,0.23419,3655.3,4461.7,9.3448,27.008,42.481,...,53.670,23.350,61.061,20.719,40.999,38.653,47.386,47.528,40.212,17.659
15329996,4999996,20.0,500.0,497,0.26704,3647.4,4540.2,9.3546,27.034,42.671,...,54.650,26.362,60.020,20.263,41.579,33.624,47.536,47.647,41.199,18.741
15329997,4999997,20.0,500.0,498,0.26543,3630.3,4571.6,9.4089,27.129,42.470,...,54.274,26.521,59.824,20.189,41.505,40.967,52.437,47.802,41.302,23.199
15329998,4999998,20.0,500.0,499,0.27671,3655.7,4498.9,9.3781,27.353,42.281,...,53.506,26.781,62.818,20.453,40.208,40.957,47.628,48.086,40.510,15.932


In [3]:
# preprocess data
df["faultNumber"] = df["faultNumber"].astype(int)
df = df.drop(["simulationRun", "sample", "index"], axis=1)

In [4]:
# remove 3, 9 and 15
mask = ~df["faultNumber"].isin([3, 9, 15])

In [5]:
df = df[mask]
df["faultNumber"].unique()

array([ 0,  1,  2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 16, 17, 18, 19,
       20])

In [6]:
# features
X = df.loc[:, df.columns != "faultNumber"].values
# labels
y = df["faultNumber"].values

X.shape, y.shape

((13140000, 52), (13140000,))

In [7]:
df.columns[df.columns != "faultNumber"]

Index(['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
       'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
       'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
       'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
       'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
       'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
       'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
       'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9',
       'xmv_10', 'xmv_11'],
      dtype='object')

### Scaler
The recommended way (see 'Elements of Statistical Learning', chapter 'The Wrong and Right Way to Do Cross-validation') is to calculate the **mean** and the **standard deviation** of the values in the **training set** and then **apply them for standardizing both the training and testing sets**.

The idea behind this is to preven**t data leakage** from the testing to the training set because the aim of model validation is to subject the testing data to the same conditions as the data used for the model training.

In [8]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X.shape

(13140000, 52)

In [9]:
scaled_df = pd.DataFrame(scaled_X, columns=df.columns[df.columns != "faultNumber"])
scaled_df.head(3)

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.048403,0.172463,-0.370896,0.317190,0.674038,0.295625,-0.255480,-0.073291,0.133912,-0.109065,...,-0.023099,-0.276875,-0.466629,-0.071480,0.052982,0.688629,0.655752,-0.156733,-0.000211,-0.485108
1,-0.044249,-0.510475,0.523796,0.069667,0.422894,0.638373,-0.260356,0.202214,-0.273659,-0.090802,...,-0.191197,-0.279855,-0.380834,-0.067129,0.039000,-0.476264,-0.587066,-0.145792,-0.130463,-0.397765
2,-0.070227,-0.490122,-0.010922,-0.245500,0.111128,-0.295842,-0.281081,-0.227164,-0.273659,-0.102173,...,-0.090887,-0.260867,-0.360861,-0.083757,0.047548,1.299152,-0.793705,-0.147038,-0.064116,0.136149


In [10]:
scaled_df.describe()

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
count,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,...,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07,1.314000e+07
mean,-7.793533e-16,-7.748663e-14,2.111692e-15,1.472170e-14,-1.295664e-13,1.536178e-13,-2.976028e-14,4.304707e-14,1.713684e-12,-4.899861e-15,...,-1.218418e-14,-8.172694e-16,1.650911e-14,-1.045025e-15,4.457157e-15,-1.731363e-14,6.869924e-15,1.264669e-15,3.776892e-15,3.278473e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.740879e+00,-8.058969e+00,-8.458078e+00,-6.959772e+00,-6.726066e+00,-8.248797e+00,-3.812835e+00,-1.005158e+01,-1.073464e+01,-3.762003e+00,...,-6.492572e+00,-1.473142e+00,-7.243393e+00,-1.816945e+00,-2.855521e+00,-5.364186e+00,-5.558764e+00,-2.755571e+00,-3.588298e+00,-2.352624e+00
25%,-2.778506e-01,-6.325895e-01,-3.394423e-01,-3.047835e-01,-5.990048e-01,-5.718597e-01,-3.505700e-01,-3.180223e-01,-2.736590e-01,-2.699887e-01,...,-1.584009e-01,-3.899846e-01,-3.967221e-01,-1.437417e-01,-8.705024e-02,-6.947449e-01,-6.641828e-01,-2.670049e-01,-1.289475e-01,-2.688179e-01
50%,-5.512817e-02,-2.201628e-02,3.930875e-03,-7.474204e-02,-5.783974e-03,-7.442083e-02,-2.554797e-01,6.812522e-02,-1.945255e-03,-1.143486e-01,...,-9.376873e-02,-2.452922e-01,-2.602000e-01,-8.973954e-02,3.639215e-02,2.637937e-02,-4.895167e-02,-1.139454e-01,-7.539861e-02,-1.264895e-01
75%,1.561219e-01,5.953411e-01,3.560413e-01,1.917818e-01,6.047572e-01,4.715487e-01,-1.396645e-01,4.220327e-01,2.697684e-01,2.877147e-02,...,-2.748989e-02,-7.411293e-02,-1.013214e-01,-4.265276e-02,1.483161e-01,6.474515e-01,6.757770e-01,9.879972e-02,-1.081993e-02,2.763879e-02
max,5.000698e+00,5.470881e+00,5.828169e+00,7.228061e+00,7.203797e+00,6.907922e+00,3.339668e+00,9.041109e+00,8.285323e+00,5.453893e+00,...,6.221384e+00,3.361250e+00,4.107940e+00,5.970003e+00,4.209672e+00,5.506107e+00,5.879678e+00,2.728975e+00,4.926123e+00,9.814238e+00


In [11]:
def create_samples(X, y, lookback=5):
    x_out = []
    y_out = []
    with tqdm(total=len(X)-lookback-1) as pbar:
        for i in range(len(X)-lookback-1):
            _x = X[i:i+lookback, :]
            _y = y[i+lookback+1]
            x_out.append(_x)
            y_out.append(_y)
            pbar.update(1)

    print("Preparing numpy return. This could take some seconds.")
    return np.array(x_out), np.array(y_out)

In [23]:
_x, _y = create_samples(scaled_df.values.astype(np.float32), y)
_x.shape, _y.shape

100%|█████████████████████████████████████████████████████████████████| 13139994/13139994 [00:05<00:00, 2503312.21it/s]


Preparing numpy return. This could take some seconds.


((13139994, 5, 52), (13139994,))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(_x, _y, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [14]:
# Train samples
print("Train:")
print(f"All samples: {X_train.shape[0]}")
print(f"Without anomalies: {X_train[y_train == 0].shape[0]}")
# Test samples
print("Test:")
print(f"All samples: {X_test.shape[0]}")
print(f"Without anomalies: {X_test[y_test == 0].shape[0]}")
# Train samples
print("Validation:")
print(f"All samples: {X_val.shape[0]}")

Train:
All samples: 9197995
Without anomalies: 511426
Test:
All samples: 1971000
Without anomalies: 109288
Validation:
All samples: 1970999


# LSTM - binary classification

In [36]:
class BinaryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0):
        super(BinaryLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=False
        )
        self.linear1 = nn.Linear(hidden_size, 1)

    def forward(self, X):
        output, (hidden_state, cell_state) = self.lstm(X)
        output = self.linear1(output[:, -1, :])
        output = torch.sigmoid(output)
        return output

# Training

In [44]:
# binary labels
to_binary_label = np.vectorize(lambda x: 1 if x > 0 else 0)
y_train_binary = to_binary_label(y_train)
y_test_binary = to_binary_label(y_test)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
class TEP(Dataset):
    def __init__(self, x, y):
        super(TEP, self).__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        _x = self.x[idx]
        _y = self.y[idx]
        return _x, _y

In [26]:
train_data = TEP(X_train, y_train_binary)
trainloader = DataLoader(train_data, batch_size=256, shuffle=True)

In [45]:
test_data = TEP(X_test, y_test_binary)
testloader = DataLoader(test_data, batch_size=256)

In [95]:
class EarlyStopping:
    def __init__(self, tolerance=3, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False
        self.best_accuracy = None

    def __call__(self, test_accuracy):
        if self.best_accuracy is None:
            self.best_accuracy = test_accuracy
        else:
            if (test_accuracy - self.min_delta) > self.best_accuracy:
                self.best_accuracy = test_accuracy
            else:
                self.counter += 1
                if self.counter > self.tolerance:
                    self.early_stop = True
                    print("Early stopping!")

In [ ]:
num_epochs = 20

# model
learning_rate = 0.0002
model = BinaryLSTM(input_size=52, hidden_size=128, num_layers=12, dropout=0.2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = nn.BCELoss()

early_stopper = EarlyStopping()

best_acc = 0
train_losses = []
test_losses = []
train_accs = []
test_accs = []
for i in range(num_epochs):
    train_epoch_losses = []
    test_epoch_losses = []
    with tqdm(total=len(trainloader)) as pbar:
        model.train()
        for x, y in trainloader:
            x = x.to(torch.float32).to(device)
            y = y[:, None]
            y = y.to(torch.float32).to(device)
            pred = model(x)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss = loss.detach().item()
            train_epoch_losses.append(loss)
            train_preds = torch.concat([train_preds, torch.round(pred.detach().cpu())])
            train_true = torch.concat([train_true, y.cpu()])
            
            pbar.update(1)
        train_acc = (pred.round() == y).float().mean()
        
        model.eval()
        for x, y in testloader:
            x = x.to(torch.float32).to(device)
            y = y[:, None]
            y = y.to(torch.float32).to(device)
            with torch.no_grad():
                pred = model(x)
            loss = loss_fn(pred, y)
            loss = loss.detach().item()
            test_epoch_losses.append(loss)
            test_preds = torch.concat([test_preds, torch.round(pred.detach().cpu())])
            test_true = torch.concat([test_true, y.cpu()])
        test_acc = (pred.round() == y).float().mean()

        # evaluations
        e_train = np.mean(train_epoch_losses)
        e_test = np.mean(test_epoch_losses)
        train_losses.append(e_train)
        test_losses.append(e_test)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
                
        print(f"Epoch {i+1} - train loss: {e_train} train acc: {train_acc} \
            \t test loss: {e_test} test acc: {test_acc}")

        if test_acc > best_acc:
            torch.save(model.state_dict(), "./model/binary_lstm/model.pt")

        early_stopper(test_acc)
        if early_stopper.early_stop:
            break

  6%|████▊                                                                        | 2225/35930 [02:08<31:42, 17.72it/s]